# **Imports and defining variables**

In [1]:
import pandas as pd
import re
import requests
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

In [7]:
navigator = 'Microsoft Edge (Scrapping; This is my system)'
url = 'https://www.auto-data.net/en'

In [8]:
html = requests.get(url, headers={'User-Agent': navigator})
html

<Response [200]>

# **Scraping**

## Creating the soup and cleaning the information to get only text

In [9]:
soup = BeautifulSoup(html.text, 'html.parser')

In [10]:
brands = soup.find_all('a', {'class' : 'marki_blok'})
brands[1]

<a class="marki_blok" href="/en/alfa-romeo-brand-11" title="Alfa Romeo - Technical Specs, Fuel consumption, Dimensions"><img alt="Alfa Romeo - Technical Specs, Fuel consumption, Dimensions" src="/img/logos/Alfa_Romeo.png"/><strong>Alfa Romeo</strong></a>

In [56]:
#This loop is to get only the part to use in link ie whatever is after 'href='
brand_list = re.findall('href="(\S+)"',str(brands))
brand_list[0]

'/en/acura-brand-6'

In [54]:
#Looping to remove the /en since the url already has it and removing it makes it so it will default to another language
brand_list_final = []
for i in range(len(brand_list)):
    brand_list_final += [re.findall('en/(\S+)', brand_list[i])]
brand_list_final

[['acura-brand-6'],
 ['alfa-romeo-brand-11'],
 ['alpina-brand-16'],
 ['aston-martin-brand-36'],
 ['audi-brand-41'],
 ['bentley-brand-66'],
 ['bmw-brand-86'],
 ['bugatti-brand-106'],
 ['cadillac-brand-121'],
 ['chevrolet-brand-156'],
 ['chrysler-brand-161'],
 ['citroen-brand-166'],
 ['cupra-brand-256'],
 ['dacia-brand-181'],
 ['daewoo-brand-191'],
 ['daihatsu-brand-2'],
 ['dodge-brand-32'],
 ['ds-brand-198'],
 ['ferrari-brand-62'],
 ['fiat-brand-67'],
 ['ford-brand-72'],
 ['gmc-brand-97'],
 ['great-wall-brand-107'],
 ['honda-brand-127'],
 ['hummer-brand-137'],
 ['hyundai-brand-147'],
 ['infiniti-brand-152'],
 ['jaguar-brand-3'],
 ['jeep-brand-8'],
 ['kia-brand-23'],
 ['koenigsegg-brand-28'],
 ['lada-brand-140'],
 ['lamborghini-brand-38'],
 ['lancia-brand-43'],
 ['land-rover-brand-48'],
 ['lexus-brand-58'],
 ['lotus-brand-78'],
 ['maserati-brand-108'],
 ['mazda-brand-118'],
 ['mclaren-brand-123'],
 ['mercedes-benz-brand-138'],
 ['mg-brand-153'],
 ['mini-brand-168'],
 ['mitsubishi-brand-1

## Creating the loops for the actual scraping

In [22]:
for urls in brand_list:
    

In [ ]:
page2 = 'https://www.auto-data.net/fr/acura-cl-model-138'         #Inside each brand, find the models

In [ ]:
page3 = 'https://www.auto-data.net/fr/acura-cl-ii-generation-7196'    #Inside each model, find the correct generation/series

In [ ]:
page4 = 'https://www.auto-data.net/fr/acura-cl-ii-3.2-i-v6-24v-type-s-263hp-automatic-31069'   #This is the final page, retrieve the desired information from here

#